<a href="https://colab.research.google.com/github/EllieKallmier/sola5050-t1-2025/blob/main/Bill_calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bill Calculator

Includes:
* Bill calculation for load profiles with and without PV
* Calculates bills with flat rate, time of use and/or demand charges and includes feed in credits and daily usage charges.
* Option to add a battery with a simple algorithm set to maximise self-consumption.

In [ ]:
# @title ### Import necessary packages
# @markdown *Run this cell*

# @markdown This cell imports packages used to handle and restructure data, draw quick
# @markdown plots to sense check results and more.

# Data handling and structuring packages:
import numpy as np
import pandas as pd
import json

# Useful plotting packages:
import matplotlib.pyplot as plt
import plotly.express as px

# Package needed for a tariff calculation function to handle a specific datetime interaction:
from dateutil.relativedelta import relativedelta

# Packages that create the interactive elements of this notebook:
import ipywidgets as widgets
from ipywidgets import interactive

# Data

To calculate bill outcomes under different tariffs and solar/battery scenarios, we need to start with some load and PV data.

We also need information about the tariffs themselves. To work with the functions in the rest of the notebook, this tariff information needs to be passed to the functions in a specific format.

Luckily we have around 100 sample residential and small business tariffs already in the correct structure that you can start with.  

<br></br>
*Extra options for super keen students:*
* *We have some large commercial tariffs also structured to work with these functions - they are more complex.*
* *I've got a script that accesses the AER's public API containing current retail tariff offerings from a range of retailers in NEM states and restructures them into this format. It is also more complex, but may be of interest if you are specifically looking into retail tariffs.*

### Load in data and pre-process
It's always important when working with data to take time to assess the quality and existing structure of the data.

Pandas is a widely used Python package for data handling and can make this checking and pre-processing step quick and easy.

In [ ]:
# First read in the data itself, making sure to read the datetime data properly:
sample_load_profile = pd.read_csv(
    "https://raw.githubusercontent.com/elliekallmier/sola5050-t1-2025/main/" + \
      "tariff_data/sample_load_profiles/S0023_profile.csv",
    parse_dates=["TS"],
    index_col="TS"
)

In [ ]:
# Then we can look at a sample from the start of this dataset:
sample_load_profile.head()

In [ ]:
# Now we can use some pandas functions to get an overview of our data:
sample_load_profile.info()

In [ ]:
# And some statistics for the numeric columns:
sample_load_profile.describe()

In [ ]:
# We will need to do some transforms to get this into the right units and structure,
# as well as filling in some of the empty points.

# We can also drop columns that aren't useful here to save space.
load_profile = sample_load_profile.drop(columns=["CUSTOMER_ID"])

# For this example: fill empty ("NaN") values with 0.0.
# NOTE: you need to come up with an appropriate and justifiable way of handling missing
# data if you come across this in your analysis.
load_profile_kwh = load_profile.fillna(0.0)

# Convert to kWh:
load_profile = load_profile * 1e-3
load_profile = load_profile.rename(columns={"Wh": "kWh"})

# And resample to half-hourly intervals:
load_profile = load_profile.resample("30min").sum()

# We also only want to analyse one year of load data in this example:
load_profile = load_profile[load_profile.index.year == 2020]
load_profile.head()

In [ ]:
# @title ### Charts

# @markdown Another great way to quickly understand your data is to plot it.
# @markdown Plotly is a great plotting package that allows you to make interactive
# @markdown charts so you can explore your data easily.

# @markdown Matplotlib is the underlying package that you can use to make fully customised
# @markdown charts.

timeseries_chart = px.line(
    load_profile,
    x=load_profile.index,
    y=["kWh", "PV"],
    title="Interactive plotly chart: Load and PV",
    labels={
        "kWh": "Load",
        "PV": "PV",
        "TS": "DateTime",
        "value":"kWh"
    }
)
timeseries_chart.show()

## Bring in tariffs

In [ ]:
# @title ## Existing tariff set

# @markdown Now we can load in all of the pre-formatted tariffs.
# @markdown We can filter these tariffs to choose only types that we are interested
# @markdown in, by customer type (residential or small business), tariff type (e.g.
# @markdown flat rate or time of use), state, or DNSP.

# @markdown Once you run this cell, a table containing all possible options for tariff
# @markdown type, state, customer type and DNSP will be printed.

# @markdown *Run this cell*
import requests

file_url = "https://raw.githubusercontent.com/elliekallmier/sola5050-t1-2025/main/tariff_data/residential_and_smallbusiness_2024_25.json"

# Fetch the JSON from the URL
response = requests.get(file_url)

all_tariffs = response.json()
all_tariffs = all_tariffs[0]["Tariffs"]

all_types = pd.Series(list(set([tar["Type"] for tar in all_tariffs])), name="Type")
all_dnsps = pd.Series(list(set([tar["Distributor"] for tar in all_tariffs])), name="Distributor")
all_states = pd.Series(list(set([tar["State"] for tar in all_tariffs])), name="State")
all_customers = pd.Series(list(set([tar["CustomerType"] for tar in all_tariffs])), name="CustomerType")

filter_details = pd.concat([all_types, all_dnsps, all_states, all_customers], axis="columns")
filter_details = filter_details.fillna("-")
filter_details

In [ ]:
# @title #### Filtering to choose tariffs {"run":"auto","display-mode":"form"}
# @markdown You can use the following inputs to filter the tariffs by field to use
# @markdown in the bill calculation.

DNSP = "CitiPower" # @param ["None", 'Powercor', 'Jemena', 'Western Power', 'Energex', 'PowerWater', 'United Energy', 'CitiPower', 'EvoEnergy', 'Ergon Energy', 'Endeavour Energy', 'Essential Energy', 'Ausgrid', 'SA Power Networks', 'Ausnet Services', 'TasNetworks', 'Horizon Power']
State = "None" # @param ["None", 'TAS', 'SA', 'NSW', 'ACT', 'NT', 'QLD', 'VIC', 'WA']
Type = "None" # @param ["None","ToU","Block","ToU_Demand","SToU","Demand","Single Rate"]
CustomerType = "Residential" # @param ["None", "Residential", "SmallBusiness"]

# @markdown Then we can use the inputs above to filter the list of tariffs and only
# @markdown collect the one(s) we want in a new list, `selected_tariffs`.

# @markdown *Run this cell*
selected_tariffs = []

for tariff in all_tariffs:
  if DNSP != "None":
    if tariff["Distributor"] != DNSP:
      continue

  if State != "None":
    if tariff["State"] != State:
      continue

  if Type != "None":
    if tariff["Type"] != Type:
      continue

  if CustomerType != "None":
    if tariff["CustomerType"] != CustomerType:
      continue

  if "Feed" in tariff["Type"]:
    continue

  selected_tariffs.append(tariff)

In [ ]:
# @markdown You can look at the selected tariff(s) in neater format by using the
# @markdown `pprint` package.

# @markdown *Run this cell to print `selected_tariffs`*
import pprint

pprint.pprint(selected_tariffs)

# Calculation Functions

This section holds all the functions that are used to calculate the energy usage or demand charges and total bill outcomes for a given load profile and chosen tariff.

You will need to run the cell below to make sure the functions are defined and usable in the next steps, but you don't need to read through or understand these functions.

In [ ]:
# @title ### Define tariff calculation functions {"display-mode":"form"}
# @markdown *Run this cell*


def time_select(
    load_profile_s: pd.DataFrame, tariff_component_details: dict
) -> pd.DataFrame:
    """Filters a load profile DataFrame based on specified time intervals and days
    of the week/month from tariff component details.

    Args:
        load_profile_s: A DataFrame containing the load profile data with
             a DateTime index.
        tariff_component_details: A dictionary containing the time intervals, weekdays,
            weekends, and months for filtering. The dictionary must have the following key/value
            pairs
            - TimeIntervals: A dictionary where each key is an interval ID
                and each value is a list of two time strings (start and end).
                Time strings in 'TimeIntervals' should be in 'HH:MM' format.
                Time intervals starting at '24:00' are adjusted to '00:00'
                for proper filtering.
            - Weekday: A boolean indicating whether weekdays are included in this
                tariff component.
            - Weekend: A boolean indicating whether weekends are included in this
                tariff component.
            - Month: A list of integers representing the months included in this
                component (e.g., [1, 2, 3] for January, February, March).
            Dict structure looks like:
            tariff_component_details = {
                "Month": [
                    1,
                    2,
                    12
                ],
                "TimeIntervals": {
                    "T1": [
                        "22:00",
                        "07:00"
                    ]
                },
                "Weekday": true,
                "Weekend": false
            }

    Returns:
        load_profile_selected_times: A DataFrame filtered to
            include only the rows that fall within the specified time intervals,
            and match the specified weekday/weekend and month criteria for the
            given tariff component.

    """
    load_profile_selected_times = pd.DataFrame()
    for (
        interval_id,
        times,
    ) in tariff_component_details["TimeIntervals"].items():
        if times[0][0:2] == "24":
            times[0] = times[1].replace("24", "00")
        if times[1][0:2] == "24":
            times[1] = times[1].replace("24", "00")
        if times[0] != times[1]:
            lp_between_times = load_profile_s.between_time(
                start_time=times[0], end_time=times[1], inclusive="right"
            )
        else:
            lp_between_times = load_profile_s.copy()

        if not tariff_component_details["Weekday"]:
            lp_times_and_days = lp_between_times.loc[
                lp_between_times.index.weekday >= 5
            ].copy()
        elif not tariff_component_details["Weekend"]:
            lp_times_and_days = lp_between_times.loc[
                lp_between_times.index.weekday < 5
            ].copy()
        else:
            lp_times_and_days = lp_between_times.copy()
        lp_times_days_months = lp_times_and_days.loc[
            lp_times_and_days.index.month.isin(tariff_component_details["Month"]), :
        ].copy()

        load_profile_selected_times = pd.concat(
            [load_profile_selected_times, lp_times_days_months]
        )
    return load_profile_selected_times


def calculate_daily_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the total daily charges for a bill.

    Args:
        load_profile: A DataFrame containing the load profile data in kWh
            with a DateTime index.
        tariff_component: A dictionary containing tariff details. It should
            include a 'Daily' key with a nested dictionary that has a 'Value' key
            specifying the daily charge rate as follows:
            tariff_component = {
                ...
                'Daily' : {
                    'Unit' : '$/Day',
                    'Value' : 10.0
                }
                ...
            }
        results: dict, not used here, included to simplify control logic.
        tariff_category: A string representing the tariff category, one of 'NUOS'
            or 'Retailer' (not used here, included to simplify control logic).

    Returns:
        float: The bill's total daily charge in dollars ($), calculated as
            num_days_in_load_profile * daily_charge_value.

    """
    num_days = len(load_profile.index.normalize().unique()) - 1
    daily_charge = num_days * tariff_component["Daily"]["Value"]
    return daily_charge


def calculate_fixed_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Returns the total fixed charges for a bill.

    Args:
        load_profile: A DataFrame containing the load profile data
            with a DateTime index (not used here, included to simplify control
            logic).
        tariff_component: A dictionary containing tariff details. It should
            include a 'Fixed' key with a nested dictionary that has a 'Value' key
            specifying the fixed rate per bill as follows:
            tariff_component = {
                ...
                'Fixed' : {
                    'Unit' : '$/Bill',
                    'Value' : 100.0
                }
                ...
            }
            - 'Unit' must be '$/Bill'
        results: dict, not used here, included to simplify control logic.
        tariff_category: A string representing the tariff category, one of 'NUOS'
            or 'Retailer' (not used here, included to simplify control logic).

    Returns:
        float: The bill's total fixed charge in dollars ($).

    """
    return tariff_component["Fixed"]["Value"]


def calculate_flatrate_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the total of all flat rate charges for a bill.

    Args:
        load_profile: DataFrame not used here, included to simplify control logic.
        tariff_component: A dictionary containing tariff details. It should
            include a 'FlatRate' key with a nested dictionary that has a 'Value' key
            specifying the daily charge rate as follows:
            tariff_component = {
                ...
                'FlatRate' : {
                    'Unit' : '$/kWh',
                    'Value' : 0.55
                }
                ...
            }
        results: A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key tariff_category with a pd.DataFrame that stores tariff component
            results.
            Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                tariff_category : pd.DataFrame()
            }
        tariff_category: str, not used here, included to simplify control logic.

    Returns:
        float: The bill's total daily charge in dollars ($), calculated as
            num_days_in_load_profile * daily_charge_value.

    """
    flat_rate_charge = (
        results["load_information"]["Annual_kWh"]
        * tariff_component["FlatRate"]["Value"]
    )
    return flat_rate_charge


def calculate_annual_block_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the total of all annual block charges for a bill.

    For each block described in the tariff component, energy usage is compared
    against the bounds of the block. Usage up to the upper bound of the block
    is charged at the block's set rate, and the remaining energy use is charged
    under the next block's rate (and so on). For example, with an annual usage
    of 1000kWh and an upper bound of 800kWh for the first block at $0.5/kWh
    and no upper bound for the second block at $0.8/kWh, the annual charge
    is calculated as 800 * 0.5 + 200 * 0.8 = $560.

    Args:
        load_profile: DataFrame not used here, included to simplify control
            logic.
        tariff_component: A dictionary containing tariff details. It should
            include a 'BlockAnnual' key with a nested dictionary with the following
            structure:
            tariff_component = {
            ...
                'BlockAnnual' : {
                    'Block1' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.20,
                        'HighBound' : 60
                    },
                    'Block2' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.55,
                        'HighBound' : Infinity
                    },
                    ...
                }
                ...
            }
        results: A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key tariff_category with a pd.DataFrame that stores tariff component
            results. Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                tariff_category : pd.DataFrame()
            }
        tariff_category: str, not used here, included to simplify control logic.

    Returns:
        float: The bill's total daily charge in dollars ($), calculated as
            num_days_in_load_profile * daily_charge_value.

    """
    block_use = results["load_information"][["Annual_kWh"]].copy()
    block_use_charge = block_use.copy()
    lim = 0
    for k, v in tariff_component["BlockAnnual"].items():
        block_use[k] = block_use["Annual_kWh"]
        block_use[k][block_use[k] > float(v["HighBound"])] = float(v["HighBound"])
        block_use[k] = block_use[k] - lim
        block_use[k][block_use[k] < 0] = 0
        lim = float(v["HighBound"])
        block_use_charge[k] = block_use[k] * v["Value"]
    del block_use["Annual_kWh"]
    del block_use_charge["Annual_kWh"]
    annual_block_charge = block_use_charge.sum(axis=1)

    return annual_block_charge


def calculate_quarterly_block_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the quarterly block charge based on the load profile and
    tariff component details.

    This function calculates quarterly consumption for each of the four quarters,
    applies the block tariff charges based on consumption within each block, and
    sums up the charges for each quarter. This total charge is returned as a float.

    Args:
        load_profile (pd.DataFrame): A DataFrame containing half-hourly timeseries
            data with a DateTime index. It should have at least one column named 'kWh'
            containing energy usage (load) values for the corresponding half-hour
            up to index.
        tariff_component (dict): A dictionary containing tariff details. It should
            include a 'BlockQuarterly' key with a nested dictionary where each
            key represents a block and each value is a dictionary with 'HighBound'
            and 'Value' specifying the upper bound and charge rate for that block:
            tariff_component = {
            ...
                'BlockQuarterly' : {
                    'Block1' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.20,
                        'HighBound' : 60
                    },
                    'Block2' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.55,
                        'HighBound' : Infinity
                    },
                    ...
                }
                ...
            }
        results (dict): A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key <tariff_category> with a pd.DataFrame that stores tariff component
            results. Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                <tariff_category> : pd.DataFrame()
            }

        tariff_category (str): A string representing the tariff category, used
            to store the charges in the results dictionary.

    Returns:
        float: The total quarterly block charge calculated from the load profile
            and tariff component details.

    Notes:
        - Quarterly periods are defined as:
            Q1: January - March
            Q2: April - June
            Q3: July - September
            Q4: October - December
    """
    # first: get quarterly consumption and save in the results 'load_information' df:
    for Q in range(1, 5):
        lp_quarterly = load_profile.loc[
            load_profile.index.month.isin(list(range((Q - 1) * 3 + 1, Q * 3 + 1))), :
        ]
        results["load_information"]["kWh_Q" + str(Q)] = [
            np.nansum(lp_quarterly[col].values[lp_quarterly[col].values > 0])
            for col in lp_quarterly.columns
        ]

    # then get the charge for each quarter:
    for Q in range(1, 5):
        block_use = results["load_information"][["kWh_Q" + str(Q)]].copy()
        block_use_charge = block_use.copy()
        lim = 0
        for k, v in tariff_component["BlockQuarterly"].items():
            block_use[k] = block_use["kWh_Q" + str(Q)]
            block_use[k][block_use[k] > float(v["HighBound"])] = float(v["HighBound"])
            block_use[k] = block_use[k] - lim
            block_use[k][block_use[k] < 0] = 0
            lim = float(v["HighBound"])
            block_use_charge[k] = block_use[k] * v["Value"]
        del block_use["kWh_Q" + str(Q)]
        del block_use_charge["kWh_Q" + str(Q)]

        results[tariff_category]["C_BlockQuarterly_" + str(Q)] = block_use_charge.sum(
            axis=1
        )

    quarterly_block_charge = results[tariff_category][
        ["C_BlockQuarterly_" + str(Q) for Q in range(1, 5)]
    ].sum(axis=1)
    return quarterly_block_charge


def calculate_monthly_block_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the monthly block charge based on the load profile and
    tariff component details.

    This function calculates consumption within each month, applies the block tariff
    charges based on consumption within each block, and sums up the charges for each
    month. This total charge is returned as a float.

    Args:
        load_profile (pd.DataFrame): A DataFrame containing half-hourly timeseries
            data with a DateTime index. It should have at least one column named 'kWh'
            containing energy usage (load) values for the corresponding half-hour
            up to index.
        tariff_component (dict): A dictionary containing tariff details. It should
            include a 'BlockMonthly' key with a nested dictionary where each
            key represents a block and each value is a dictionary with 'HighBound'
            and 'Value' specifying the upper bound and charge rate for that block:
            tariff_component = {
            ...
                'BlockMonthly' : {
                    'Block1' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.20,
                        'HighBound' : 60
                    },
                    'Block2' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.55,
                        'HighBound' : Infinity
                    },
                    ...
                }
                ...
            }
        results (dict): A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key <tariff_category> with a pd.DataFrame that stores tariff component
            results. Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                <tariff_category> : pd.DataFrame()
            }

        tariff_category (str): A string representing the tariff category, used
            to store the charges in the results dictionary.

    Returns:
        float: The total monthly block charge calculated from the load profile
            and tariff component details.

    """
    # Get monthly consumtion and store in results 'load_information' df:
    for m in range(1, 13):
        lp_monthly = load_profile.loc[load_profile.index.month == m, :]
        results["load_information"]["kWh_m" + str(m)] = [
            np.nansum(lp_monthly[col].values[lp_monthly[col].values > 0])
            for col in lp_monthly.columns
        ]

    # then calculate the charge for each month:
    for m in range(1, 13):
        block_use = results["load_information"][["kWh_m" + str(m)]].copy()
        block_use_charge = block_use.copy()
        lim = 0
        for k, v in tariff_component["BlockMonthly"].items():
            block_use[k] = block_use["kWh_m" + str(m)]
            block_use[k][block_use[k] > float(v["HighBound"])] = float(v["HighBound"])
            block_use[k] = block_use[k] - lim
            block_use[k][block_use[k] < 0] = 0
            lim = float(v["HighBound"])
            block_use_charge[k] = block_use[k] * v["Value"]
        del block_use["kWh_m" + str(m)]
        del block_use_charge["kWh_m" + str(m)]
        results[tariff_category]["C_BlockMonthly_" + str(m)] = block_use_charge.sum(
            axis=1
        )

    monthly_block_charge = results[tariff_category][
        ["C_BlockMonthly_" + str(m) for m in range(1, 13)]
    ].sum(axis=1)
    return monthly_block_charge


def calculate_daily_block_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the daily block charge based on the load profile and
    tariff component details.

    This function calculates consumption within each month, applies the block tariff
    charges based on consumption within each block, and sums up the charges for each
    month. This total charge is returned as a float.

    Args:
        load_profile (pd.DataFrame): A DataFrame containing half-hourly timeseries
            data with a DateTime index. It should have at least one column named 'kWh'
            containing energy usage (load) values for the corresponding half-hour
            up to index.
        tariff_component (dict): A dictionary containing tariff details. It should
            include a 'BlockDaily' key with a nested dictionary where each
            key represents a block and each value is a dictionary with 'HighBound'
            and 'Value' specifying the upper bound and charge rate for that block:
            tariff_component = {
            ...
                'BlockDaily' : {
                    'Block1' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.20,
                        'HighBound' : 60
                    },
                    'Block2' : {
                        'Unit' : '$/kWh',
                        'Value' : 0.55,
                        'HighBound' : Infinity
                    },
                    ...
                }
                ...
            }
        results (dict): dict, not used here, included to simplify control logic.
        tariff_category (str): str, not used here, included to simplify control logic.

    Returns:
        float: The total daily block charge calculated from the load profile
            and tariff component details.

    """

    # First, resample the load profile to get daily usage:
    daily_kwh_usage = load_profile.resample("D").sum()
    block_use_temp_charge = daily_kwh_usage.copy()
    block_use_temp_charge.iloc[:, :] = 0
    lim = 0
    # then apply the daily blocks to find daily charges:
    for block, details in tariff_component["BlockDaily"].items():
        block_use_temp = daily_kwh_usage.copy()
        block_use_temp[block_use_temp > float(details["HighBound"])] = float(
            details["HighBound"]
        )
        block_use_temp = block_use_temp - lim
        block_use_temp[block_use_temp < 0] = 0
        lim = float(details["HighBound"])
        block_use_temp_charge = (
            block_use_temp_charge + block_use_temp * details["Value"]
        )
    daily_block_charge = block_use_temp_charge.sum(axis=0)
    return daily_block_charge


def calculate_time_of_use_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculates the total time of use energy charge based on the load profile and
    tariff component details.

    This function calculates consumption within each defined time of use period,
    applies the tariff rate based on consumption within each period, and sums up
    all time of use charges.

    Args:
        load_profile (pd.DataFrame): A DataFrame containing half-hourly timeseries
            data with a DateTime index. It should have at least one column named 'kWh'
            containing energy usage (load) values for the corresponding half-hour
            up to index.
        tariff_component (dict): A dictionary containing tariff details. It should
            include a 'TOU' key with a nested dictionary where each key represents
            a charging period and each value is a dictionary with details specifying
            month, time and weekdays during which the charge applies, as well as
            the units ($/kWh) and rate of the charge itself.
        results (dict): A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key <tariff_category> with a pd.DataFrame that stores tariff component
            results. Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                <tariff_category> : pd.DataFrame()
            }
        tariff_category (str): A string representing the tariff category, used
            to store the charges in the results dictionary.

    Returns:
        float: The total TOU charge calculated from the load profile and tariff
            component details.

    """
    # First set up temporary dfs to hold interim results:
    time_of_use_consumption = pd.DataFrame()
    time_of_use_consumption_charge = pd.DataFrame()
    # Loop over each TOU component (e.g. Peak, Weekend Off-Peak, Shoulder etc)
    # and fill in any missing details with default values
    for tou_component, details in tariff_component["TOU"].items():
        details_copy = details.copy()
        if "Weekday" not in details_copy:
            details_copy["Weekday"] = True
            details_copy["Weekend"] = True
        if "TimeIntervals" not in details_copy:
            details_copy["TimeIntervals"] = {"T1": ["00:00", "00:00"]}
        if "Month" not in details_copy:
            details_copy["Month"] = list(range(1, 13))

        # Then call time_select to get the load_profile for times during which
        # this charge component applies. Calculate usage then total charge for
        # this period:
        lp_time_of_use = time_select(load_profile, details_copy)
        time_of_use_consumption[tou_component] = lp_time_of_use.sum()
        results[tariff_category]["kWh_" + tou_component] = time_of_use_consumption[
            tou_component
        ].copy()
        time_of_use_consumption_charge[tou_component] = (
            details_copy["Value"] * time_of_use_consumption[tou_component]
        )
        results[tariff_category]["TOU_" + tou_component] = (
            time_of_use_consumption_charge[tou_component].copy()
        )

    time_of_use_charge = time_of_use_consumption_charge.sum(axis=1)
    return time_of_use_charge


def calc_dem_(
    dem_component_details: dict,
    num_peaks: int,
    load_profile_selected_times: pd.DataFrame,
    tariff_category: str,
    demand_component: str,
    results: dict,
) -> float:
    """Calculate the demand charge based on demand component details and load profile data.

    This function computes the demand charge based on the provided demand component details,
    the number of peaks to consider, and the load profile. It updates a results DataFrame
    with the average demand and the total demand charge for the given tariff category and
    demand component.

    Args:
        dem_component_details: A dictionary containing details about the demand component,
            such as minimum demand and charge values. Expected keys are 'Value', 'Unit' ($/kW/day),
            'Min Demand (kW)' and 'Min Demand Charge ($)'.
        num_peaks: The number of peaks to consider when calculating the demand charge.
        load_profile_selected_times: DataFrame containing the load profile with
            datetime index and at least one column named 'kWh' containing half-hourly
            load data. This dataframe will contain load data for selected periods
            based on the tariff component, calculated before being passed to this function.
        tariff_category: A string representing the tariff category, used
            to store the charges in the results dictionary.
        demand_component:A string naming the demand charge component, used
            to store the charges in the results dictionary.
        results:A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key <tariff_category> with a pd.DataFrame that stores tariff component
            results.

    Returns:
        float: The total demand charge calculated based on the demand component details and load
            profile data.

    Notes:
        - The function updates the `results[<tariff_category>]` DataFrame with two new columns for the specified
          `demand_component`:
            - 'Avg_kW_Dem_<demand_component>': The average demand in kW.
            - 'Demand_<demand_component>': The total demand charge in dollars.
    """

    # Get any value(s) for min demand present in the tariff definition:
    min_demand = 0
    min_demand_from_charge = 0
    if "Min Demand (kW)" in dem_component_details:
        min_demand = dem_component_details["Min Demand (kW)"]
    if "Min Demand Charge ($)" in dem_component_details:
        if dem_component_details["Value"] > 0:
            min_demand_from_charge = (
                dem_component_details["Min Demand Charge ($)"]
                / dem_component_details["Value"]
            )

    # Set up an empty array to hold peak values:
    average_peaks_all = np.empty((0, load_profile_selected_times.shape[1]), dtype=float)

    # Loop through each month present in the tariff component definition to find
    # peaks
    for m in dem_component_details["Month"]:
        arr = (
            load_profile_selected_times.loc[
                load_profile_selected_times.index.month == m, :
            ]
            .copy()
            .values
        )
        arr.sort(axis=0)
        arr = arr[::-1]

        # 2 * -> to change units from kWh to kW. Get the average of the peaks (if
        # the number of peaks is > 1)
        average_peaks_all = np.append(
            average_peaks_all, [2 * arr[:num_peaks, :].mean(axis=0)], axis=0
        )

    # If there is a minimum demand set in the tariff component, depending on the
    # type of minimum set, apply here:
    if min_demand_from_charge > 0:
        # If the minimum demand comes from min_demand_from_charge, apply it as
        # a clipping value
        average_peaks_all = np.clip(
            average_peaks_all, a_min=min_demand_from_charge, a_max=None
        )
    else:
        # Otherwise, if it's coming from min_demand (or is <= 0), any 'peak' value
        # less than the min_demand is set to zero.
        average_peaks_all[average_peaks_all < min_demand] = 0

    # Sum all average peaks form each month together and use this sum to calculate
    # the total demand charge for this bill.
    average_peaks_all_sum = average_peaks_all.sum(axis=0)
    results[tariff_category]["Avg_kW_Dem_" + demand_component] = (
        average_peaks_all_sum / len(dem_component_details["Month"])
    )
    results[tariff_category]["Demand_" + demand_component] = (
        average_peaks_all_sum * dem_component_details["Value"] * 365 / 12
    )  # the charges in demand charge should be in $/kW/day

    dem_charge = average_peaks_all_sum * dem_component_details["Value"] * 365 / 12

    return dem_charge


def calculate_demand_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculate the total demand charge for all `Demand` tariff components.

    This function acts as a wrapper for `calc_dem()`, passing each component of a
    `Demand` tariff individually to calculate the charge for that component only.
    It also calls `time_select()` to pass only relevant parts of the load profile
    to calculate the demand charge.

    Args:
        load_profile (pd.DataFrame): A DataFrame containing half-hourly timeseries
            data with a DateTime index. It should have at least one column named 'kWh'
            containing energy usage (load) values for the corresponding half-hour
            up to index.
        tariff_component (dict): A dictionary containing tariff details. It should
            include a `Demand` key with a nested dictionary where each key represents
            a demand period and each value is a dictionary with details specifying
            month, time and weekdays during which the charge applies, as well as
            the units ($/kW/day) and rate of the charge itself.
        results (dict): A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key <tariff_category> with a pd.DataFrame that stores tariff component
            results. Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                <tariff_category> : pd.DataFrame()
            }
        tariff_category (str): A string representing the tariff category, used
            to store the charges in the results dictionary.

    Returns:
        float: The sum of demand charges calculated based on the sum of component
            charges.

    """
    demand_charge_total = 0.0
    for demand_component, demand_component_details in tariff_component[
        "Demand"
    ].items():
        if "Number of Peaks" not in demand_component_details:
            num_of_peaks = 1
        else:
            num_of_peaks = demand_component_details["Number of Peaks"]

        lp = load_profile.copy()
        lp_selected_times = time_select(lp, demand_component_details)

        demand_charge_total += calc_dem_(
            demand_component_details,
            num_of_peaks,
            lp_selected_times,
            tariff_category,
            demand_component,
            results,
        )

    return demand_charge_total


def calculate_off_peak_demand_charge(
    load_profile: pd.DataFrame,
    tariff_component: dict,
    results: dict,
    tariff_category: str,
) -> float:
    """Calculate the total demand charge for all `Off-Peak Demand` tariff components.

    This function acts as a wrapper for `calc_dem()`, first compiling the load profile
    during all off-peak periods by repeatedly calling `time_select()` for each
    `Off-Peak Demand` component, then passing the resulting load profile to calculate
    the total off-peak demand charge.

    Args:
        load_profile (pd.DataFrame): A DataFrame containing half-hourly timeseries
            data with a DateTime index. It should have at least one column named 'kWh'
            containing energy usage (load) values for the corresponding half-hour
            up to index.
        tariff_component (dict): A dictionary containing tariff details. It should
            include an `Off_Peak Demand` key with a nested dictionary where each
            key represents a demand period and each value is a dictionary with details
            specifying month, time and weekdays during which the charge applies, as
            well as the units ($/kW/day) and rate of the charge itself.
        results (dict): A dict containing key 'load_information' with a pd.DataFrame
            value that has column 'Annual_kWh' with a single entry at index
            'kWh' that holds the annual energy usage of the given load profile,
            and key <tariff_category> with a pd.DataFrame that stores tariff component
            results. Structured as follows:
            results = {
                'load_information' : pd.DataFrame(
                    columns=['Annual_kWh'],
                    index=['kWh'],
                    data=[6758021.922]
                ),
                <tariff_category> : pd.DataFrame()
            }
        tariff_category (str): A string representing the tariff category, used
            to store the charges in the results dictionary.

    Returns:
        float: The total sum of off-peak demand charges.

    """
    lp_off_peak_dem = pd.DataFrame()
    for opd_component, opd_component_details in tariff_component[
        "Off Peak Demand"
    ].items():
        lp = load_profile.copy()
        lp_selected_times = time_select(lp, opd_component_details)
        lp_off_peak_dem = pd.concat([lp_off_peak_dem, lp_selected_times], axis="rows")

    if "Number of Peaks" not in opd_component_details:
        num_of_peaks = 1
    else:
        num_of_peaks = opd_component_details["Number of Peaks"]

    demand_charge = calc_dem_(
        opd_component_details,
        num_of_peaks,
        lp_off_peak_dem,
        tariff_category,
        "Off_Peak",
        results,
    )

    return demand_charge


# -------------------- Bill Calculator function for residential and small business tariffs -----------
def bill_calculator(load_profile, tariff, fit=True):
    """
    Calculate the billing charges for residential or small busiess tariffs based on
    the load profile and tariff details.

    This function computes the energy bill for a residential or small busiess load profile, including
    daily, fixed, flat rate, block, time-of-use (TOU), demand, and excess demand charges. It also
    handles retailer-specific tariff adjustments and calculates energy charges. The results are
    stored in a dictionary with detailed billing information for each tariff component.

    This function was originally written for the following tools and has been adapted
    for use in SunSPOT.
    - CEEM Bill_calculator github - https://github.com/UNSW-CEEM/Bill_Calculator
    - CEEM tariff tool github - https://github.com/UNSW-CEEM/TDA_Python

    Args:
        load_profile: DataFrame containing the load profile data for one year.
            It should have two columns: 'TS' (timestamp) and 'kWh' (kilowatt-hours).
        tariff: Dictionary containing tariff details, including tariff parameters and
            types of charges.
        network_load: optional, DataFrame containing network load data. This
            parameter is not used in this function but included for consistency with other tools.
        fit: optional, flag indicating whether to include the Feed-in Tariff (FiT) rebate
            in the calculations. Defaults to True.

    Returns:
        results: A dictionary containing billing results for each tariff component. The dictionary
            includes
                - `'load_information'`: DataFrame with annual consumption information.
                - `'Retailer'`: DataFrame with charges calculated for each component present in the
                    chosen tariff.

    Notes:
        - The function uses a dictionary of functions to calculate different types of charges based
          on the tariff parameters.
        - If the tariff provider is a retailer, it adjusts the tariff parameters accordingly.
        - The function handles a variety of tariff charge types and calculates the total bill and
          energy charges for each tariff component.
        - The 'load_information' DataFrame in the results dictionary provides annual kWh consumption data
          and, if applicable, the annual kWh exported.
        - The function assumes that the load profile data is provided in half-hourly intervals.

    Examples:
        >>> load_profile = pd.DataFrame({
        ... 'TS': pd.date_range(start='2023-01-01', periods=8760, freq='30T'),
        ... 'kWh': np.random.rand(8760)})
        >>> tariff = {
        ... "CustomerType": "Residential",
        ... "Date_accessed": "2024-02",
        ... "Distributor": "Ausgrid",
        ... "Name": "Example Tariff",
        ... "Parameters": {<tariff_details>},
        ... "ProviderType": "Retailer",
        ... "State": "VIC",
        ... "Tariff ID": "ID01",
        ... "Type": "TOU",
        ... "Year": "2024"}

        >>> results = bill_calculator_residential_small_business_tariffs(load_profile, tariff)
        >>> results['load_information']
                Annual_kWh  Annual_kWh_exp
        kWh     6758021.92            -0.0

        >>> results['bill_outcomes']
            Charge_Daily  ...         Bill    energy_charge
        kWh     68860.43  ... 1.469779e+06        202740.66

    """
    load_profile = load_profile[["kWh", "PV"]].copy().fillna(0.0)
    load_profile["Net Load"] = load_profile["kWh"] - load_profile["PV"]
    load_profile = load_profile.rename(columns={"kWh":"Load", "Net Load": "kWh"})
    load_profile = pd.DataFrame(load_profile["kWh"])

    ## Set up "results" dictionary to store calculated consumption and tariff
    ## charges.
    results = {}

    # Calculate imports and exports
    temp_import = load_profile.values
    temp_export = temp_import.copy()
    temp_import[temp_import < 0] = 0
    temp_export[temp_export > 0] = 0

    lp_net_import = pd.DataFrame(
        temp_import, columns=load_profile.columns, index=load_profile.index
    )
    lp_net_export = pd.DataFrame(
        temp_export, columns=load_profile.columns, index=load_profile.index
    )

    results["load_information"] = pd.DataFrame(
        index=[col for col in load_profile.columns],
        data=np.sum(lp_net_import.values, axis=0),
        columns=["Annual_kWh"],
    )

    if fit:
        results["load_information"]["Annual_kWh_exp"] = -1 * np.sum(
            lp_net_export.values, axis=0
        )

    # If it is retailer put retailer as a component to make it similar to network tariffs
    if tariff["ProviderType"] == "Retailer":
        tariff_temp = tariff.copy()
        del tariff_temp["Parameters"]
        tariff_temp["Parameters"] = {"bill_outcomes": tariff["Parameters"]}
        tariff = tariff_temp.copy()

    func_dict = {
        "Daily": (calculate_daily_charge, "Charge_Daily"),
        "Fixed": (calculate_fixed_charge, "Charge_Fixed"),
        "FlatRate": (calculate_flatrate_charge, "Charge_FlatRate"),
        "BlockAnnual": (calculate_annual_block_charge, "Charge_BlockAnnual"),
        "BlockQuarterly": (calculate_quarterly_block_charge, "Charge_BlockQuarterly"),
        "BlockMonthly": (calculate_monthly_block_charge, "Charge_BlockMonthly"),
        "BlockDaily": (calculate_daily_block_charge, "Charge_BlockDaily"),
        "TOU": (calculate_time_of_use_charge, "Charge_TOU"),
        "Demand": (calculate_demand_charge, "Charge_Demand"),
        "Off Peak Demand": (calculate_off_peak_demand_charge, "Charge_Off_Peak_Demand")
    }

    # Set up another entry to results dict to contain charge/bill results for
    # the network component (called "Retailer" for Large Comms for consistency)
    # with small business/residential.
    for component_type, component_details in tariff["Parameters"].items():
        results[component_type] = pd.DataFrame(index=results["load_information"].index)
        results[component_type]["Charge_FiT_Rebate"] = 0

        # Calculate the FiT
        if "BlockDailyFiT" in component_details.keys():
            DailyExportkWh = lp_net_export.resample("D").sum()
            DailyExportkWh["kWh"] = -1 * DailyExportkWh["kWh"]
            block_use_temp_charge = DailyExportkWh.copy()
            block_use_temp_charge.iloc[:, :] = 0
            lim = 0
            for k, v in component_details["BlockDailyFiT"].items():
                block_use_temp = DailyExportkWh.copy()
                block_use_temp[block_use_temp > float(v["HighBound"])] = float(
                    v["HighBound"]
                )
                block_use_temp = block_use_temp - lim
                block_use_temp[block_use_temp < 0] = 0
                lim = float(v["HighBound"])
                block_use_temp_charge = (
                    block_use_temp_charge + block_use_temp * v["Value"] * -1
                )
            results[component_type]["Charge_FiT_Rebate"] = block_use_temp_charge.sum(
                axis=0
            )
        elif "FiT_TOU" in component_details.keys():
            load_profile_ti_exp = pd.DataFrame()
            load_profile_ti_exp_charge = pd.DataFrame()
            for k, v in component_details["FiT_TOU"].items():
                this_part = v.copy()
                if "Weekday" not in this_part:
                    this_part["Weekday"] = True
                    this_part["Weekend"] = True
                if "TimeIntervals" not in this_part:
                    this_part["TimeIntervals"] = {"T1": ["00:00", "00:00"]}
                if "Month" not in this_part:
                    this_part["Month"] = list(range(1, 13))
                load_profile_t_a = time_select(lp_net_export, this_part)
                load_profile_ti_exp[k] = load_profile_t_a.sum()
                results[component_type]["kWh_Exp" + k] = load_profile_ti_exp[k].copy()
                load_profile_ti_exp_charge[k] = (
                    this_part["Value"] * load_profile_ti_exp[k]
                )
                results[component_type]["FiT_C_TOU" + k] = load_profile_ti_exp_charge[
                    k
                ].copy()
            results[component_type]["Charge_FiT_Rebate"] = (
                load_profile_ti_exp_charge.sum(axis=1)
            )
        elif "FiT" in component_details.keys():
            results[component_type]["Charge_FiT_Rebate"] = (
                -1
                * results["load_information"]["Annual_kWh_exp"]
                * component_details["FiT"]["Value"]
            )

        # Loop through each charge component in the tariff (e.g. TOU, Demand)
        # and calculate the amount to be charged under this component
        for charge_type in component_details.keys():
            if "FiT" not in charge_type:
                results[component_type][func_dict[charge_type][1]] = func_dict[
                    charge_type
                ][0](lp_net_import, component_details, results, component_type)

    energy_comp_list = [
        "BlockAnnual",
        "BlockQuarterly",
        "BlockMonthly",
        "BlockDaily",
        "FlatRate",
        "TOU",
    ]

    for k, v in results.items():
        if k != "load_information":
            results[k]["total_bill"] = results[k][
                [col for col in results[k].columns if col.startswith("Charge")]
            ].sum(axis=1)
            results[k]["energy_charges"] = results[k][
                [
                    col
                    for col in results[k].columns
                    if (
                        col.startswith("Charge")
                        and col.endswith(tuple(energy_comp_list))
                    )
                ]
            ].sum(axis=1)
            results[k]["demand_charges"] = results[k][
                [
                    col
                    for col in results[k].columns
                    if (col.startswith("Charge") and col.endswith("Demand"))
                ]
            ].sum(axis=1)

    return results

### Run tariff calculations on sample load profile

In [ ]:
# First just the load on its own:
all_bill_results = []
for tariff in selected_tariffs:
  load_profile_no_pv = load_profile.copy()
  load_profile_no_pv["PV"] = 0.0

  all_calculations = bill_calculator(load_profile_no_pv, tariff)

  bill_outcomes = all_calculations["bill_outcomes"]

  bill_outcomes.index = [tariff['Name']]
  bill_outcomes["Extras"] = "None"

  all_bill_results.append(bill_outcomes)


bills_no_solar = pd.concat(all_bill_results, axis="rows")
bills_no_solar[["Charge_FiT_Rebate", "Charge_Daily", "energy_charges", "demand_charges", "total_bill", "Extras"]]

In [ ]:
# Then with solar:
all_bill_results_with_pv = []
for tariff in selected_tariffs:
  all_calculations_pv = bill_calculator(load_profile, tariff)
  bill_outcomes_pv = all_calculations_pv["bill_outcomes"]
  bill_outcomes_pv.index = [tariff['Name']]
  bill_outcomes_pv["Extras"] = "Solar"
  all_bill_results_with_pv.append(bill_outcomes_pv)

bills_with_solar = pd.concat(all_bill_results_with_pv, axis="rows")
bills_with_solar[["Charge_FiT_Rebate", "Charge_Daily", "energy_charges", "demand_charges", "total_bill", "Extras"]]

In [ ]:
# @title ### Define simple battery algorithm
# @markdown *Run this cell*


# -------------------- Battery Calculator functions -----------
def battery(profiles_b, battery_kw, battery_kwh, battery_eff=0.90, del_t=2):
    """
    Simulates a simple battery algorithm to maximize self-consumption of PV energy.

    The function charges the battery using excess PV generation that is not consumed by the load and
    discharges the battery to minimize grid import. It updates the state of charge (SOC) and net load
    accordingly.

    Args:
        profiles_b (pd.DataFrame): DataFrame containing PV and Load columns (in kWh).
        battery_kw (float): Battery power rating (kW).
        battery_kwh (float): Battery energy capacity (kWh).
        battery_eff (float, optional): Battery round-trip efficiency (default is 0.90).
        del_t (int, optional): Time step duration in intervals per hour (default is 2,
          representing 30min intervals).

    Returns:
        pd.DataFrame: Updated DataFrame with added columns for battery state of charge (SOC),
        excess PV, excess load, excess charge, and net load.
    """

    profiles_fr = profiles_b.copy() * del_t # convert to kW from kWh
    profiles_fr = profiles_fr.reset_index().rename(columns={"kWh": "Load"})
    profiles_fr['SOC'] = 0
    profiles_fr['ExcessPV'] = (profiles_fr['PV'] - profiles_fr['Load']).clip(lower=0, upper=battery_kw)
    profiles_fr['ExcessLoad'] = (profiles_fr['Load'] - profiles_fr['PV']).clip(lower=0, upper=battery_kw)
    for i in range(1, len(profiles_fr)):
        profiles_fr.loc[i, 'SOC'] = max(0, min(battery_kwh, profiles_fr.loc[i - 1, 'SOC'] +
                                                (battery_eff ** 0.5) * profiles_fr.loc[i, 'ExcessPV'] / del_t -
                                                profiles_fr.loc[i, 'ExcessLoad'] / (
                                                        battery_eff ** 0.5) / del_t))
        profiles_fr.loc[i, 'ExcessCharge'] = profiles_fr.loc[i, 'SOC'] - profiles_fr.loc[i - 1, 'SOC']

    profiles_fr['ExcessCharge'] = profiles_fr['ExcessCharge'].apply(
        lambda x: x * (battery_eff ** 0.5) if x < 0 else x / (battery_eff ** 0.5))

    profiles_fr["Charging"] = profiles_fr["ExcessCharge"].clip(lower=0) * del_t
    profiles_fr["Discharging"] = profiles_fr["ExcessCharge"].clip(upper=0) * del_t

    profiles_fr['Net Load'] = profiles_fr['Load'] - profiles_fr['PV'] + profiles_fr['ExcessCharge'] * del_t
    profiles_fr["Load with battery"] = profiles_fr['Load'] + profiles_fr['Discharging']
    profiles_fr["PV with battery"] = profiles_fr['PV'] - profiles_fr['Charging']

    profiles_b = profiles_fr[['TS', "Net Load", "Load with battery", "PV with battery", 'SOC']].set_index('TS')
    profiles_b.rename(columns={'Load with battery':"kWh", 'PV with battery':"PV"}, inplace=True)
    profiles_b['kWh'] = profiles_b['kWh'] / del_t
    profiles_b['PV'] = profiles_b['PV'] / del_t
    profiles_b['Net Load'] = profiles_b['Net Load'] / del_t

    return profiles_b


In [ ]:
# Calling the battery algorithm is pretty straightforward:
load_with_battery = battery(load_profile, battery_kw=8, battery_kwh=8, battery_eff=0.95)

# And we can run through the bill calculator with all the tariffs again:
all_bill_results_with_battery = []
for tariff in selected_tariffs:
  all_calculations_battery = bill_calculator(load_with_battery[["kWh", "PV"]], tariff)
  bill_outcomes_battery = all_calculations_battery["bill_outcomes"]
  bill_outcomes_battery.index = [tariff['Name']]
  bill_outcomes_battery["Extras"] = "Battery"
  all_bill_results_with_battery.append(bill_outcomes_battery)

bills_with_solar_and_battery = pd.concat(all_bill_results_with_battery, axis="rows")
bills_with_solar_and_battery[["Charge_FiT_Rebate", "Charge_Daily", "energy_charges", "demand_charges", "total_bill", "Extras"]]

In [ ]:
# Now lets combine all the bill results and compare:

combined_bill_results = pd.concat([bills_no_solar, bills_with_solar, bills_with_solar_and_battery], axis="rows")

combined_bill_results = combined_bill_results[["energy_charges", "demand_charges", "total_bill", "Extras"]].copy().reset_index()
melted = combined_bill_results.melt(id_vars=["index", "Extras"], var_name="Result", value_name="$")

bar_plot = px.bar(
    melted,
    x="Result",
    y="$",
    color="Extras",
    facet_col="index",
    barmode="group"
)
bar_plot.show()

In [ ]:
load_with_battery["Net Load without Battery"] = (load_profile["kWh"] - load_profile["PV"])
timeseries_chart = px.line(
    load_with_battery,
    x=load_with_battery.index,
    y=["kWh", "PV", "Net Load", "Net Load without Battery"],
    title="Interactive plotly chart: Load, PV and Net Load",
    labels={
        "kWh": "Load",
        "PV": "PV",
        "TS": "DateTime",
        "value":"kWh"
    }
)
timeseries_chart.show()

Other notes to touch on:

- More tools: PPA Tool (need emissions data - I can share some)
  - But includes more complex battery optimisation if interested
- ISP inputs - we have packages to make them easier to read/process (advanced)
- Notebook with dispatch simulation for timeseries: also advanced but we can share/help.